## Enderscopy stage demo

### This notebook shows how to connect and drive an enderscope stage from Python.

In [2]:
import time
from enderscope import SerialUtils, Stage

In [3]:
# list available serial ports
ports = SerialUtils.serial_ports()
print (ports)

['/dev/tty.Bluetooth-Incoming-Port', '/dev/tty.usbmodem144301']


In [4]:
stageport = ports[1]

In [5]:
# connect to enderscope and create a stage object
s = Stage(stageport, 115200)

In [6]:
# home the stage for positionning reproducibility
s.home()

In [7]:
# move to (x,y(,z)) absolute coordinate
s.move_absolute(10,10,10)

In [9]:
# sending specific low level GCode to read device parameters
s.write_code(f"M114", debug=True)

X:10.00 Y:10.00 Z:10.00 E:0.00 F:2000.00 move:90
M114


'ok\r\n'

In [8]:
# this can be used to set axis speeds:
s.write_code(f"M203 Z50") # increase z axis speed to 50
s.move_axis('z',40) # go up fast
s.write_code(f"M203 Z5") # restore z axis speed to 5
s.move_axis('z',-40) # go down slowly

In [9]:
# follow arbitrary curve from GCode
# create curve in inkscape, export as svg, import in slicer, export gcode
# then read it back in a string
import os
def read_g_code(fname):
    fpath = os.path.join(".", fname)
    with open(fpath, "r") as text_file:
        data = text_file.read()
    return data
path = read_g_code("tests/path2.gcode")

In [10]:
print(path)

G1 X39.133 Y60.578 E.11078
G1 X34.751 Y59.885 E.12542
G1 X29.685 Y58.899 E.14591
G1 X27.082 Y58.291 E.07557
G1 X24.533 Y57.604 E.07463
G1 X22.121 Y56.839 E.07154
G1 X19.932 Y55.998 E.0663
G1 X18.023 Y55.073 E.05997
G1 X16.475 Y54.064 E.05224
G1 X15.861 Y53.529 E.02302
G1 X15.368 Y52.974 E.02099
G1 X15.005 Y52.399 E.01922
G1 X14.769 Y51.77 E.01899
G1 X14.593 Y50.62 E.03289
G1 X14.636 Y49.518 E.03118
G1 X14.883 Y48.465 E.03058
G1 X15.293 Y47.499 E.02967
G1 X15.841 Y46.583 E.03018
G1 X16.499 Y45.707 E.03097
G1 X18.018 Y44.041 E.06374
G1 X19.629 Y42.412 E.06477
G1 X21.087 Y40.751 E.06248
G1 X21.675 Y39.899 E.02927
G1 X22.146 Y38.998 E.02874
G1 X22.463 Y38.051 E.02823
G1 X22.598 Y37.047 E.02864
G1 X22.523 Y36.038 E.0286
G1 X22.256 Y35.087 E.02793
G1 X21.818 Y34.168 E.02878
G1 X21.256 Y33.305 E.02912
G1 X19.876 Y31.656 E.06079
G1 X18.339 Y30.017 E.06352
G1 X16.905 Y28.32 E.06281
G1 X16.301 Y27.418 E.03069
G1 X15.817 Y26.471 E.03007
G1 X15.474 Y25.442 E.03066
G1 X15.323 Y24.393 E.02996
G1 X15

In [11]:
s.set_absolute()
# multiline GCode is not supported yet so cut it in pieces for now
for i in range(1,50):
    bit = path.split('\n')[i]
    bit = bit[0:bit.index('E')-1]
    s.write_code(bit, debug=False)